Данное задание основано на материалах лекций по линейной регрессии и посвящено предсказанию оклада, исходя из описания вакансии.



Инструкция по выполнению

- Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).
- Проведите предобработку:
    - Приведите тексты к нижнему регистру (text.lower()).
    - Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.
    - Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
    - Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.
    - Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
    - Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.
- Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.
- Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.



In [1]:
import pandas as pd
import sklearn

# Load data

In [16]:
data_train = pd.read_csv('data/salary-train.csv')

In [17]:
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


# Preprocessing

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

In [44]:
import scipy

In [48]:
tfidf = TfidfVectorizer(min_df=5)
enc = DictVectorizer()

In [49]:
def prep(data, tfidf, enc, new=False):
    data['LocationNormalized'].fillna('nan', inplace=True)
    data['ContractTime'].fillna('nan', inplace=True)
    data['FullDescription'] = data['FullDescription'].str.lower()
    data['FullDescription'] = data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
    if new:
        tfidf.fit(data.FullDescription)
        enc.fit(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
    
    descriptions = tfidf.transform(data.FullDescription)
    X_train_categ = enc.transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
    X = scipy.sparse.hstack([descriptions, X_train_categ])
    y = data['SalaryNormalized']
    return X, y

In [50]:
X_train, y_train = prep(data_train, tfidf, enc, new=True)

In [51]:
X_train

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [52]:
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


# Train

In [53]:
from sklearn.linear_model import Ridge

In [54]:
regr = Ridge(alpha=1, random_state=241)

In [55]:
regr.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

# Predict

In [56]:
data_test = pd.read_csv('data/salary-test-mini.csv')

In [57]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [58]:
X_test, y_test = prep(data_test, tfidf, enc, new=False)

In [59]:
X_test

<2x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [60]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [61]:
y_test.head()

0   NaN
1   NaN
Name: SalaryNormalized, dtype: float64

In [67]:
ans = ' '.join(['{:.2f}'.format(round(y, 2)) for y in regr.predict(X_test)])
ans

'56555.62 37188.32'

In [68]:
with open('t01-lin-regression-ans.txt', 'w') as fd:
    fd.write(ans)